### Import statements

In [14]:
import pyodbc
import pandas as pd
from IPython.display import display, Markdown

### Connect to server with dummy data

In [15]:
server = 'covid.ebmdatalab.net,1433'
database = 'OPENCoronaExport' 
username = 'SA'
password = 'ahsjdkaJAMSHDA123[' 
cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()

### Find the smoking codes from QoF

In [16]:
qof = pd.read_csv('../data/QoFClusteres_CTV3Codes - Sheet1.csv')

#### Import function

In [17]:
from lib.data_gathering import search_terms_to_df

### Search Qof codes

In [18]:
smoking_list = ['SMOK', 'CESS']

In [19]:
smoking_qof = search_terms_to_df(smoking_list, col_name='ClusterId', df_name=qof)

In [20]:
smoking_qof.head()

,ClusterId,ClusterName,CTV3Code,Description
3495,CESS,Smoking cessation codes,6791.,Health education - smoking
3496,CESS,Smoking cessation codes,67A3.,Pregnancy smoking advice
3497,CESS,Smoking cessation codes,9OO1.,Attends stop smoking monitoring
3498,CESS,Smoking cessation codes,9OO2.,Refuses stop smoking monitor
3499,CESS,Smoking cessation codes,9OO3.,Stop smoking monitor default


#### Output codes to csv

In [21]:
smoking_qof.to_csv('../data/smoking_codes.csv')

### Checking

There would need to be some sort of manual checking of these codes

### Find people with these codes in TPP data

In [22]:
from lib.data_gathering import codes_to_sql_where

In [23]:
codes_where = codes_to_sql_where("CTV3Code", smoking_qof['CTV3Code'])

In [24]:
# NBVAL_IGNORE_OUTPUT
query = f'''
SELECT * --DISTINCT Patient_ID
FROM CodedEvent
WHERE ({codes_where}) AND ConsultationDate BETWEEN '2015-01-01' AND '2020-03-31'
'''
df = pd.read_sql(query, cnxn, index_col='Patient_ID')
df.head()

,Consultation_ID,CodedEvent_ID,CTV3Code,NumericValue,ConsultationDate,SnomedConceptId
Patient_ID,,,,,,
685,2757948190,2198985768,Ua1Nz,0.0,2019-10-01 12:29:08,225323000
731,2757977414,2199085869,Ua1Nz,0.0,2019-10-01 12:49:20,225323000
827,2757977563,2199085918,Ua1Nz,0.0,2019-10-01 12:49:20,225323000
1009,2757977336,2199085720,Ua1Nz,0.0,2019-10-01 12:49:20,225323000
1009,2757948092,2198685838,Ua1Nz,0.0,2019-10-01 12:29:08,225323000


In [25]:
cnxn.close()